# Law

In [ ]:
import sys

# append the directory of law module to sys.path list
sys.path.append('../../modules/')

In [ ]:
import datetime as dt
import glob
import math
import os
import re

import arrest
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

## NIBRS

In [ ]:
nibrs_file = '../../US/01_inputs/FBI/N-DEx-IEPD-4.0/documentation/nibrs/N-DEx_4.0_CodeTables_NIBRS.xlsx'

In [ ]:
nibrs = pd.ExcelFile(nibrs_file)

In [ ]:
nibrs_df = pd.read_excel(
    nibrs_file, sheet_name='OffenseCode', dtype=str, skiprows=2)

In [ ]:
nibrs_df.dropna(axis=1, how='all', inplace=True)

In [ ]:
nibrs_df['NCIC Code'].fillna('', inplace=True)

In [ ]:
fbi_offense_categories = pd.read_csv(
    '../../US/01_inputs/processed/fbi_offense_categories.csv',
    dtype=str,
    # usecols=['_offense_category_code', '_offense_category', '_crime_against'],
    keep_default_na=False,
)

## CIBRS

In [ ]:
cibrs_file = pd.ExcelFile('../01_inputs/CIBRS Offense Listing.xlsx')

In [ ]:
cibrs_file.sheet_names

In [ ]:
cibrs_df = pd.read_excel(
    '../01_inputs/CIBRS Offense Listing.xlsx', sheet_name='Offense Codes', dtype=str)

In [ ]:
cibrs_subset = cibrs_df[['CJIS CODE', 'NIBRS CODES']].copy()

In [ ]:
cibrs_subset['NIBRS Offense Code'] = cibrs_subset['NIBRS CODES'].str.split(',')

In [ ]:
cibrs_subset = cibrs_subset.explode('NIBRS Offense Code')

In [ ]:
cibrs_subset[['CJIS CODE', 'NIBRS Offense Code']].drop_duplicates()

In [ ]:
nibrs_categories = pd.read_csv(
    '../../US/01_inputs/processed/nibrs_categories.csv', dtype=str)

In [ ]:
nibrs_df = pd.merge(
    cibrs_subset[['CJIS CODE', 'NIBRS Offense Code']].drop_duplicates(), nibrs_categories)

In [ ]:
cjis_nibrs_df = nibrs_df.groupby('CJIS CODE').agg(
    {'Crime Against': arrest.format_unique, 'NIBRS Offense Description': arrest.format_unique, 'NIBRS Offense Category': arrest.format_unique}).reset_index()

In [ ]:
cjis_nibrs_df.rename(columns={'CJIS CODE': 'CJISCode'}, inplace=True)

## `CHSOFF`

- [Data source](https://oag.ca.gov/law/code-tables) (specifically [Offense Codes (with LEI codes)](https://oag.ca.gov/sites/all/files/agweb/law-enforcement/code-tables/chsoff.csv?041820220129))
- [Names Source (XML)](https://oag.ca.gov/sites/all/files/agweb/law-enforcement/code-tables/chsoff.xml)

In [ ]:
chsoff_xml = pd.read_xml('../01_inputs/chsoff_20220415.xml')

In [ ]:
chsoff_xml.columns

In [ ]:
chsoff_original = pd.read_csv(
    '../01_inputs/chsoff_20220204.csv',
    header=None,
    names=chsoff_xml.columns,
    dtype=str,
)

In [ ]:
chsoff = chsoff_original.drop(
    labels=[
        'ValidationCode',
        'TransactionTypeCode',
        'DefaultTypeofCharge',
        'LiteralIndentifierCode',
        'ChargeDegree',
        'BCSCodesLEI',
        'BCSHierarchyCodes',
        'ALPSCognizantCode',
    ],
    axis=1,
).copy()

In [ ]:
chsoff['EnactDate'] = pd.to_datetime(chsoff['EnactDate'])

In [ ]:
chsoff['RepealAmendDate'] = pd.to_datetime(
    chsoff['RepealAmendDate'], errors='coerce')

### Subset data to applicable time periods

In [ ]:
chsoff[chsoff['TypeOfStatCode'] == 'ZZ']

In [ ]:
chsoff = (
    chsoff[
        (chsoff['EnactDate'] < '2021')
        & (
            (chsoff['RepealAmendDate'] < '2020-12-31')
            | chsoff['RepealAmendDate'].isnull()
        )
        & (~chsoff['TypeOfStatCode'].isin(['ZZ']))
    ]
    .copy()
    .reset_index(drop=True)
)

### Split `StatutoryNumericCodes` into section and subparts

In [ ]:
chsoff[
    ['_section', '_subdivision', '_paragraph', '_subparagraph', '_clause']
] = chsoff.apply(
    lambda x: arrest.Charge.parse_code(
        x['StatutoryNumericCodes'], state='CA', arrests=False),
    axis=1,
    result_type='expand',
)

### Reassemble into _charge_reconstructed for joining on arrest data

In [ ]:
chsoff['_charge_reconstructed'] = chsoff['TypeOfStatCode'].str.cat(
    chsoff[['_section', '_subdivision', '_paragraph', '_subparagraph', '_clause']],
    sep='',
    na_rep='',
)

### Join on NIBRS data for later comparison

In [ ]:
chsoff_nibrs_df = pd.merge(chsoff, cjis_nibrs_df, how='inner')

In [ ]:
chsoff_nibrs_df.to_csv('../01_inputs/processed/c01_nibrs_chsoff.csv',index=False)

### Prepare for joining on arrest data

In [ ]:
chsoff.rename(
    columns={
        'StatutoryNumericCodes': '_original_code',
        'TypeOfStatCode': '_code_type',
        'TypeOfCharge': '_offense_level',
        'StatuteLiteral': '_charge_description'
    },
    inplace=True,
)

In [ ]:
chsoff.sort_values(
    by=[
        '_code_type',
        '_original_code',
        '_offense_level',
        'EnactDate',
        'RepealAmendDate',
    ],
    ascending=[True, True, True, False, False],
    inplace=True,
    na_position='first',
)

### Handle amendments

In [ ]:
repealed_amended = set(
    chsoff[chsoff['RepealAmendDate'].notnull()]['_charge_reconstructed'])

In [ ]:
not_repealed_amended = set(
    chsoff[chsoff['RepealAmendDate'].isnull()]['_charge_reconstructed'])

In [ ]:
amended = chsoff[chsoff['_charge_reconstructed'].isin(
    repealed_amended & not_repealed_amended)].copy()

#### Subset those amended in 2020 because I'll use the descriptions therein

In [ ]:
amended_late = amended[amended['RepealAmendDate'] >= '2020'].copy()

In [ ]:
not_replaced = chsoff[(chsoff['_charge_reconstructed'].isin(
    repealed_amended - not_repealed_amended))].copy()

In [ ]:
not_amended = chsoff[(chsoff['_charge_reconstructed'].isin(
    not_repealed_amended - repealed_amended))].copy()

#### Combine DataFrames

In [ ]:
processed = pd.concat(
    [amended_late, not_replaced, not_amended], ignore_index=True)

In [ ]:
processed.columns

### Identify duplicates in processed data

In [ ]:
processed_to_dedupe = processed[
    processed.duplicated(
        subset=['_charge_reconstructed', '_offense_level'], keep=False)
].copy()

In [ ]:
processed_charges = set(processed['_charge_reconstructed'])

In [ ]:
unique_processed = processed[~processed['_charge_reconstructed'].isin(
    set(processed_to_dedupe['_charge_reconstructed']))].copy()

In [ ]:
unique_processed[unique_processed.duplicated(
    subset=['_charge_reconstructed', '_offense_level'], keep=False)]

In [ ]:
unique_processed['statute_object'] = unique_processed.apply(
    lambda x: {
        'EnactDate': x['EnactDate'],
        'RepealAmendDate': x['RepealAmendDate'],
        '_charge_description': x['_charge_description'],
    },
    axis=1,
)

In [ ]:
unique_processed.columns

#### Combine duplicates in processed data with unprocessed data

In [ ]:
unprocessed_to_dedupe = chsoff[~chsoff['_charge_reconstructed'].isin(
    processed_charges)].copy()

In [ ]:
unprocessed_to_dedupe.columns

In [ ]:
to_dedupe = pd.concat(
    [processed_to_dedupe, unprocessed_to_dedupe], ignore_index=True)

In [ ]:
to_dedupe.columns

### Prepare for joinng on arrest data with missing or incorrect levels, subparts

#### Create objects with date and description data

I want to use unique charge codes while keeping this data for retrieval later.

In [ ]:
to_dedupe.sort_values(
    by=['_charge_reconstructed', 'EnactDate', 'RepealAmendDate'],
    ascending=False,
    inplace=True,
)

In [ ]:
to_dedupe['statute_object'] = to_dedupe.apply(
    lambda x: {
        'EnactDate': x['EnactDate'],
        'RepealAmendDate': x['RepealAmendDate'],
        '_charge_description': x['_charge_description'],
    },
    axis=1,
)

In [ ]:
to_dedupe.columns

In [ ]:
deduped = (
    to_dedupe.groupby(
        ['_original_code', '_code_type', '_offense_level', '_section', '_subdivision',
         '_paragraph', '_subparagraph', '_clause', '_charge_reconstructed']
    )
    .agg(_enactment_data=('statute_object', list),
         _charge_description=('_charge_description', arrest.format_unique))
    .reset_index()
)

In [ ]:
deduped[
    deduped.duplicated(
        subset=['_charge_reconstructed', '_offense_level'], keep=False)
].sort_values(by='_charge_reconstructed')

In [ ]:
chsoff_dedupe = pd.concat([deduped, unique_processed], ignore_index=True)

In [ ]:
deduped.columns

In [ ]:
chsoff_dedupe.columns

In [ ]:
chsoff_dedupe[
    [
        '_original_code',
        '_code_type',
        '_offense_level',
        '_section',
        '_charge_reconstructed',
        '_enactment_data',
        '_charge_description',
        'EnactDate',
        'RepealAmendDate',
    ]
].to_json('../01_inputs/processed/c01_chsoff.json', date_format='iso', date_unit='s')

In [ ]:
chsoff_dedupe['_level_data'] = chsoff_dedupe.apply(
    lambda x: {x['_offense_level']: x['statute_object']}, axis=1)

In [ ]:
chsoff_dedupe_without_level = (
    chsoff_dedupe.groupby(
        ['_code_type', '_section', '_charge_reconstructed'])
    .agg(_data_per_charge_level=('_level_data', list),
         _potential_offense_levels=('_offense_level', arrest.format_unique),
         _charge_description=('_charge_description', arrest.format_unique),
         )
    .reset_index()
)

In [ ]:
chsoff_dedupe_without_level[chsoff_dedupe_without_level.duplicated(
    subset=['_charge_reconstructed'], keep=False)]

In [ ]:
chsoff_dedupe_without_level.columns

In [ ]:
chsoff_dedupe_without_level[['_code_type', '_section', '_charge_reconstructed',
                             '_data_per_charge_level', '_potential_offense_levels',
                             '_charge_description']].to_json(
    '../01_inputs/processed/c01_chsoff_ignoring_level.json', date_format='iso', date_unit='s')

In [ ]:
chsoff_dedupe['_level_and_subpart_data'] = chsoff_dedupe.apply(
    lambda x: {x['_section']+x['_subdivision']: x['_level_data']}, axis=1
)

The most common issue with data entry is the omission of subdivision data altogether where the charge actually corresponds to the first subdivision of the statute.

In [ ]:
first_subdivision = chsoff_dedupe[chsoff_dedupe['_subdivision'].isin(
    ['', '(A)', '(1)'])].copy()

In [ ]:
first_subdivision['_level_quantified'] = first_subdivision['_offense_level'].replace(
    {'X': 0, 'I': 1, 'M': 2, 'F': 3})

In [ ]:
first_subdivision.sort_values(
    by=['_charge_reconstructed', '_level_quantified'], inplace=True)

#### Account for charges missing subparts, but including charge level

In [ ]:
first_subdivision_with_level = first_subdivision.drop_duplicates(
    subset=['_code_type', '_section', '_offense_level'], keep='first'
).copy()

In [ ]:
first_subdivision_with_level['_match_on_level'] = True

#### Account for charges missing both subparts and charge level

In [ ]:
first_subdivision_without_level = first_subdivision.drop_duplicates(
    subset=['_code_type', '_section'], keep='first'
)[['_original_code', '_code_type', '_section', '_charge_reconstructed',
   '_charge_description', 'statute_object', '_level_data']].copy()

In [ ]:
first_subdivision_without_level['_match_on_level'] = False

In [ ]:
ignoring_subparts = pd.concat(
    [first_subdivision_with_level, first_subdivision_without_level], ignore_index=True)

In [ ]:
ignoring_subparts[ignoring_subparts.duplicated(
    subset=['_code_type', '_section', '_offense_level'], keep=False)]

In [ ]:
ignoring_subparts.columns

In [ ]:
ignoring_subparts.drop(labels=['_subdivision', '_paragraph', '_subparagraph',
                       '_clause', '_level_quantified'], axis=1, inplace=True)

In [ ]:
ignoring_subparts[['_original_code', '_code_type', '_offense_level', '_section',
                   '_charge_reconstructed', '_enactment_data', '_charge_description', 'EnactDate', 'RepealAmendDate', 'statute_object',
                   '_level_data', '_level_and_subpart_data', '_match_on_level']].to_json(
    '../01_inputs/processed/c01_chsoff_ignoring_subparts.json',
    date_format='iso',
    date_unit='s',
)